In [1]:
import pandas as pd
import numpy as np
import random
import torch
from torch.utils.data import TensorDataset, DataLoader
from utils import load_dataset_df,  data_splitter, get_spiking_net, make_filename, smiles_to_feat
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score
from csnn_model import get_prediction_fn

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\knsve\Desktop\MEI\Tese\torch\snn_venv\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometri

#### Load DataFrame

In [2]:
files = ['tox21.csv','sider.csv', 'BBBP.csv']
dt_file = files[0]
dirname = dt_file.removesuffix('.csv')

df, targets = load_dataset_df(filename=dt_file)


In [3]:
if dirname == 'tox21':
    # SR-ARE
    target_name = targets[7]
    # SR-MMP
elif dirname == 'sider':
    #Hepatobiliary disorders 1427 samples, 0.52 class ratio
    target_name = targets[0]
else:
    target_name = targets[0]
    
df = df[[target_name, 'smiles']].dropna()

#### Molecular Representation

In [ ]:
representations = ["fp", "descriptor", "SELFIES-1hot", "SMILES-1hot"]

repr_type = representations[1]

In [5]:
if repr_type == "fp":
    fp_types = [['morgan', 1024], ['maccs', 167], ['RDKit', 1024], ['count_morgan', 1024], ['pubchem', 881]]
    mix = False
    fp_type, num_bits = fp_types[1]
    if mix and fp_type == 'RDKit':
        num_bits = 512
    data_config = {"fp_type": fp_type,
                "num_bits": num_bits,
                "radius": 2,
                "fp_type_2": fp_types[0][0],
                "num_bits_2": 1024 - num_bits,
                "mix": mix,
                "dim_2": False}
    dim_2 = data_config['dim_2']
    print(fp_type, '-', num_bits)
    if mix: print(data_config['fp_type_2'], '-', data_config['num_bits_2'])
    if dim_2: print("2D FP")

elif repr_type == "descriptor":
    desc_type = ["RDKit", "Mordred"]
    data_config = {"desc_type": desc_type[0],
                   "size": 0,
                }

data_config["repr_type"] = repr_type
print(repr_type)

descriptor


In [6]:
dtype = torch.float
split = "scaffold" if dirname == "BBBP" else "random"
dataset = None


feat_tensor, target_tensor, feat_df = smiles_to_feat(df,repr_type=repr_type, data_config=data_config, target_name=target_name, dtype=dtype)
print(feat_tensor.shape)
dataset = TensorDataset(feat_tensor, target_tensor)

[14:56:32] Explicit valence for atom # 8 Al, 6, is greater than permitted
[14:56:32] Explicit valence for atom # 3 Al, 6, is greater than permitted
[14:56:32] Explicit valence for atom # 4 Al, 6, is greater than permitted
[14:56:32] Explicit valence for atom # 4 Al, 6, is greater than permitted
[14:56:32] Explicit valence for atom # 9 Al, 6, is greater than permitted
[14:56:32] Explicit valence for atom # 5 Al, 6, is greater than permitted
[14:56:32] Explicit valence for atom # 16 Al, 6, is greater than permitted


feat_array shape: (5825, 217)
valid_df shape: (5825, 2)
Inf in descriptor MaxPartialCharge for molecule CC1=C2N=C(C=C3N=C(C(C)=C4[C@@H](CCC(N)=O)[C@](C)(CC(N)=O)[C@](C)([C@@H]5N=C1[C@](C)(CCC(=O)NC[C@@H](C)OP(=O)([O-])O[C@@H]1[C@@H](CO)O[C@H](n6cnc7cc(C)c(C)cc76)[C@@H]1O)[C@H]5CC(N)=O)N4[Co+]C#N)[C@@](C)(CC(N)=O)[C@@H]3CCC(N)=O)C(C)(C)[C@@H]2CCC(N)=O
Inf in descriptor MinPartialCharge for molecule CC1=C2N=C(C=C3N=C(C(C)=C4[C@@H](CCC(N)=O)[C@](C)(CC(N)=O)[C@](C)([C@@H]5N=C1[C@](C)(CCC(=O)NC[C@@H](C)OP(=O)([O-])O[C@@H]1[C@@H](CO)O[C@H](n6cnc7cc(C)c(C)cc76)[C@@H]1O)[C@H]5CC(N)=O)N4[Co+]C#N)[C@@](C)(CC(N)=O)[C@@H]3CCC(N)=O)C(C)(C)[C@@H]2CCC(N)=O
Inf in descriptor MaxAbsPartialCharge for molecule CC1=C2N=C(C=C3N=C(C(C)=C4[C@@H](CCC(N)=O)[C@](C)(CC(N)=O)[C@](C)([C@@H]5N=C1[C@](C)(CCC(=O)NC[C@@H](C)OP(=O)([O-])O[C@@H]1[C@@H](CO)O[C@H](n6cnc7cc(C)c(C)cc76)[C@@H]1O)[C@H]5CC(N)=O)N4[Co+]C#N)[C@@](C)(CC(N)=O)[C@@H]3CCC(N)=O)C(C)(C)[C@@H]2CCC(N)=O
Inf in descriptor MinAbsPartialCharge for molecule

#### Loss Function

In [7]:
loss_types = ['ce_mem', 'rate_loss', 'count_loss', 'temporal_loss', 'bce_loss']
loss_type = loss_types[2]
print(loss_type)


count_loss


#### Test Loop

In [8]:
net_types = ["SNN", "DSNN", "CSNN", "RSNN"]
net_type = net_types[0]
slope = 10
#spike_grad = surrogate.fast_sigmoid(slope=slope)
spike_grad = None
beta = 0.95
bias = True
net_config = {
            "num_hidden": 1024,
            "num_hidden_l2": 256,
            "num_steps": 10,
            "spike_grad": spike_grad,
            "slope": None if not spike_grad else slope, #spike_grad.__closure__[0].cell_contents,
            "beta": beta,
            "encoding": 'rate' if loss_type != 'temporal_loss' else 'ttfs',
            "bias": bias,
            "out_num": 2,
            "num_hidden_layers": 2,
            "num_hidden_l3": 256,
            }
if net_type == "CSNN":
    net_config['num_conv'] = 1
    net_config["pool_size"] = 2
    net_config["conv_kernel"] = 3
    net_config["conv_stride"] = [1 for _ in range(net_config['num_conv'])]
    net_config["conv_groups"] = 1

if repr_type == "fp":
    net_config["input_size"] = 1024 if data_config['mix'] else num_bits
    net_config["2d"] = data_config['dim_2']

elif repr_type == "descriptor":
    net_config["input_size"] = feat_tensor.shape[1]
    net_config["2d"] = False
    #net_config["num_steps"] = 10

print(net_type)

SNN


In [20]:
   
pop_coding = net_config['out_num'] > 2
lr=1e-4 #1e-6 default for 1000 epochs. csnn requires higher
iterations = 30
weight_decay = 0 # 1e-5
#weight_decay = 1e-4
optim_type = 'Adam'
#optim_type = 'SGD'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(device)
batch_size = 16 #16, 8
scores = True
train_config = {"num_epochs": 100,
                "batch_size": batch_size,
                "device": device,
                "loss_type": loss_type,
                "loss_fn": None,
                'dtype': dtype,
                'num_steps': net_config['num_steps'],
                'val_net': None,
                'prediction_fn': get_prediction_fn(encoding=net_config['encoding'], pop_coding=pop_coding, scores=scores),
                }
drop_last = net_type == "CSNN"
pin_memory = device == "cuda"
save_csv = True
save_models = True
print(train_config["prediction_fn"])

cpu
<function prediction_spk_rate_scores at 0x000002CF16FAE8C0>


In [ ]:
def calc_metrics(metrics_list, all_targets, all_preds):
    auc_roc = roc_auc_score(all_targets, all_preds)
    all_preds = np.array(all_preds) > 0.0
    
    accuracy = accuracy_score(all_targets, all_preds)
    auc_roc = roc_auc_score(all_targets, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_targets, all_preds).ravel()
    sensitivity = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = f1_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds)
    
    print("acc:", accuracy,"auc:", auc_roc)
    metrics_list[0].append(accuracy)
    metrics_list[1].append(auc_roc)
    metrics_list[2].append(sensitivity)
    metrics_list[3].append(specificity)
    metrics_list[4].append(f1)
    metrics_list[5].append(precision)
    

In [11]:
import os
import copy

net_list = []
    
net, train_net, val_net, test_net = get_spiking_net(net_type, net_config)
filename = make_filename(dirname, target_name, net_type, data_config, lr, weight_decay, optim_type, net_config, train_config, net, model = True)

model_name = filename.removesuffix('.csv')

models_path = os.path.join("results", dirname, "models", "")
all_model_names = os.listdir(models_path)
print(models_path)
#print(all_model_names)
for iter in range(iterations):
    seed = int(iter + 1)
    string_id = f"seed-{seed}.pth"
    search_name = model_name + str(string_id) 
    search_name_no_folder = search_name.removeprefix(models_path)
    if search_name_no_folder in all_model_names:
        state_dict = torch.load(search_name, weights_only=True)
        net_list.append(copy.deepcopy(state_dict))
    else: print(search_name_no_folder)



217 1024
1024 2
results\tox21\models\
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-1.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-2.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-3.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-4.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-5.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-6.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-7.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-8.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-9.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-10.pth
SR-ARE_SNN_beta-0.95_

In [15]:
if len(net_list) < 30:
    net_list = []
    for iter in range(iterations):
        seed = int(iter + 1)
        string_id = f"seed-{seed}_auc.pth"
        search_name = model_name + str(string_id) 
        search_name_no_folder = search_name.removeprefix(models_path)
        if search_name_no_folder in all_model_names:
            state_dict = torch.load(search_name, weights_only=True)
            net_list.append(copy.deepcopy(state_dict))
        else: print(search_name_no_folder)

if len(net_list) < 30:
    net_list = []
    for iter in range(iterations):
        seed = int(iter + 1)
        string_id = f"-scores-seed-{seed}.pth"
        search_name = model_name + str(string_id) 
        search_name_no_folder = search_name.removeprefix(models_path)
        if search_name_no_folder in all_model_names:
            state_dict = torch.load(search_name, weights_only=True)
            net_list.append(copy.deepcopy(state_dict))

SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-1_auc.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-2_auc.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-3_auc.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-4_auc.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-5_auc.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-6_auc.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-7_auc.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-8_auc.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-9_auc.pth
SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_biasseed-10_auc.pth
SR-ARE_SNN_beta-0.9

In [13]:
def minmax_norm(train_subset, val_subset, test_subset):
    train_tensor, _ = train_subset[:]
    val_tensor, _ = val_subset[:]
    test_tensor, _ = test_subset[:]

    min_val = train_tensor.min(dim=0).values
    max_val = train_tensor.max(dim=0).values
    range_val = (max_val - min_val).clamp(min=1e-6)

    train_norm = ((train_tensor - min_val) / range_val).clamp(0.0, 1.0)
    val_norm   = ((val_tensor   - min_val) / range_val).clamp(0.0, 1.0)
    test_norm  = ((test_tensor  - min_val) / range_val).clamp(0.0, 1.0)

    return train_norm, val_norm, test_norm

In [23]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

results = [[], [], [], [], [], []]
for iter in range(iterations):
    #print(f"Iteration:{iter + 1}/{iterations}")
    seed = iter + 1
    print(f"Seed:{seed} -> ",end='', flush=True)
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    # DATA SPLIT
    train, val, test = data_splitter(feat_df, target_name, split=split, dataset=dataset, data_config=data_config, seed=seed, dtype=dtype)
    _, train_label = train[:]
    _, val_label = val[:]
    _, test_label = test[:]

    if repr_type == "descriptor":
        train_data, val_data, test_data = minmax_norm(train, val, test)
        train = TensorDataset(train_data, train_label)
        val = TensorDataset(val_data,val_label)
        test = TensorDataset(test_data, test_label)

    #train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, pin_memory=pin_memory, drop_last=drop_last)
    #val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)

    # TESTING
    model = net
    flag = False
    try:
        loaded_result = model.load_state_dict(net_list[iter])
    except RuntimeError as e:
        print(f"{e}")
        model.load_state_dict(net_list[iter], strict=False)
        flag = True
    model.to(device)
    all_preds, all_targets = test_net(model, device, test_loader, train_config)
    calc_metrics(results, all_preds=all_preds, all_targets=all_targets)


Seed:1 -> acc: 0.7353951890034365 auc: 0.6521891274366522
Seed:2 -> acc: 0.7491408934707904 auc: 0.653262257534913
Seed:3 -> acc: 0.781786941580756 auc: 0.6674280979997633
Seed:4 -> acc: 0.7731958762886598 auc: 0.6844878048780489
Seed:5 -> acc: 0.7491408934707904 auc: 0.6631720491373613
Seed:6 -> acc: 0.7766323024054983 auc: 0.7168580212058473
Seed:7 -> acc: 0.7749140893470791 auc: 0.6660962115528882
Seed:8 -> acc: 0.7749140893470791 auc: 0.6523173479240426
Seed:9 -> acc: 0.6993127147766323 auc: 0.6716475095785441
Seed:10 -> acc: 0.788659793814433 auc: 0.6514792899408285
Seed:11 -> acc: 0.781786941580756 auc: 0.6613222304011779
Seed:12 -> acc: 0.8384879725085911 auc: 0.7033945986496624
Seed:13 -> acc: 0.761168384879725 auc: 0.6750115442971172
Seed:14 -> acc: 0.7302405498281787 auc: 0.6452574525745257
Seed:15 -> acc: 0.7852233676975945 auc: 0.6791409465020577
Seed:16 -> acc: 0.7474226804123711 auc: 0.6877760397497239
Seed:17 -> acc: 0.7405498281786942 auc: 0.6313757525570153
Seed:18 -> 

#### Save Metrics

In [24]:
metrics_np = np.zeros(12)

for i, metric in enumerate(results):
    metrics_np[i*2] = np.round(np.mean(metric), 3)
    metrics_np[i*2+1] = np.round(np.std(metric), 3)

# Print Results
print(f"Accuracy:  {metrics_np[0]:.3f} ± {metrics_np[1]:.3f}")
print(f"AUC ROC: {metrics_np[2]:.3f} ± {metrics_np[3]:.3f}")
print(f"Sensitivity: {metrics_np[4]:.3f} ± {metrics_np[5]:.3f}")
print(f"Specificity: {metrics_np[6]:.3f} ± {metrics_np[7]:.3f}")


metric_names = ['Acc', 'AUC', 'Sn', 'Sp', 'F1', 'Precision']
metrics_np = metrics_np.reshape(1, -1)
columns = []
for name in metric_names:
    columns.extend([f'Mean {name}', f'Std {name}'])


df_metrics = pd.DataFrame(metrics_np, columns=columns)
num_hidden = net_config['num_hidden']
time_steps = train_config['num_steps']
num_epochs = train_config['num_epochs']

df_raw = pd.DataFrame({name: results[i] for i, name in enumerate(metric_names)})
df_raw["Seed"] = list(range(1, 31))
df_raw = df_raw[["Seed"] + metric_names]  # reorder columns

blank = pd.DataFrame([[""] * 12] * 3, columns=columns) 
save = True
filename = make_filename(dirname, target_name, net_type, data_config, lr, weight_decay, optim_type, net_config, train_config, model)
save = not flag


Accuracy:  0.765 ± 0.029
AUC ROC: 0.666 ± 0.023
Sensitivity: 0.521 ± 0.070
Specificity: 0.810 ± 0.043


In [ ]:
filename = filename.replace("results", "results_2")
filename = filename.replace(".csv", "_no_scores.scv")
if save: 
    df_metrics.to_csv(filename, index=False)
    blank.to_csv(filename, mode='a', index=False, header=False)
    df_raw.to_csv(filename, mode='a', index=False)

print(filename)

results_2\tox21\SR-ARE_SNN_beta-0.95_desc_217_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_bias_no_scores.scv
